In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
import tqdm
import random
from lib.utils.config import load_config
from pypdf import PdfReader, PdfWriter, PdfMerger
from pdf2image import convert_from_path
import glob
import os

import cv2

from PIL import Image, ImageDraw, ImageFont

# set random seed
seed = 42
random.seed(seed)

def to_grayscale(img):
    mean = img.mean(2)
    x = np.stack([mean, mean, mean], axis=2)
    x = x.astype(np.uint8)
    return x

In [ ]:
x = cv2.imread("/home/korth/sketch2shape/data/shapenet_chair_4096/shapes/1a8bbf2994788e2743e99e0cae970928/normals/00006.png")

In [ ]:
plt.imshow(to_grayscale(x))

In [ ]:
dataset_path = "/home/korth/sketch2shape/data/shapenet_chair_4096"
import pandas as pd
metainfo = pd.read_csv(dataset_path + "/metainfo.csv")
metainfo

# Create Figures of Objects

In [ ]:
figures_path = "test_data_collection/figures"
combined_path = "test_data_collection/combined"

if not os.path.exists(combined_path):
    os.makedirs(combined_path)

if not os.path.exists(figures_path):
    os.makedirs(figures_path)

In [ ]:
test_obj_ids = metainfo[metainfo['split'] == 'test']['obj_id'].tolist()
test_obj_ids

# white image
white_image = np.ones((256, 256, 3))

options = [6, 7, 11, 12]

img_type = "normals" # sketches

for obj_id in tqdm.tqdm(test_obj_ids):
    selected_view = random.choice(options)
    sketch_path = dataset_path + f"/shapes/{obj_id}/{img_type}/000{selected_view:02d}.png"
    sketch = cv2.imread(sketch_path)

    if img_type == "normals":
        sketch = to_grayscale(sketch)

    plt.subplot(1, 2, 1)
    plt.imshow(sketch)
    plt.xticks([])
    plt.yticks([])

    # add the white image
    plt.subplot(1, 2, 2)
    plt.imshow(white_image)
    plt.xticks([])
    plt.yticks([])
    
    # add global title to the figure
    plt.suptitle(f"{obj_id}")

    # save the figure
    plt.savefig(f"test_data_collection/figures/{img_type}_{obj_id}.png")

    plt.close()


# Put Figures into PDF and Merge

In [ ]:
# Get the list of image file names in the folder
image_files = glob.glob(os.path.join(figures_path, f"{img_type}_*.png"))

# Sort the image files in ascending order
image_files.sort()

for i, image_file in enumerate(image_files):
    # Initialize an empty list to store the images
    images = []

    # Load and append each image to the list
    image = cv2.imread(image_file)
    images.append(image)

    # Combine the images vertically
    combined_image = cv2.vconcat(images)

    # Save the combined image
    combined_image_path = combined_path + f"/combined_image_{img_type}_{i}.png"
    cv2.imwrite(combined_image_path, combined_image)
    cv2.destroyAllWindows()

# Define the font properties for the page number
font = ImageFont.truetype("/usr/share/fonts/truetype/ubuntu/Ubuntu-R.ttf", 12)

# Iterate over the image files
for i, image_file in enumerate(image_files):
    # Open the image
    image = Image.open(image_file)

    # Create a new blank image with white background
    page_image = Image.new("RGB", (image.width, image.height + 20), "white")

    # Paste the original image onto the new image
    page_image.paste(image, (0, 0))

    # Create a drawing object
    draw = ImageDraw.Draw(page_image)

    # Calculate the position for the page number
    page_number_position = (10, image.height + 5)

    # Draw the page number on the image
    draw.text(page_number_position, f"{i+1}", font=font, fill="black")

    # Save the page as a PDF
    pdf_path = combined_path + f"/combined_image_{img_type}_{i}.pdf"
    page_image.save(pdf_path, "PDF", resolution=100.0)
    

# Create a PdfMerger object
pdf_merger = PdfMerger()

# Iterate over the PDF files
for i, image_file in enumerate(image_files):
    # Add the PDF to the merger object
    pdf_merger.append(combined_path + f"/combined_image_{img_type}_{i}.pdf")
    
# Save the combined PDF to disk
combined_pdf_path = combined_path + f"/combined_image_{img_type}.pdf"
with open(combined_pdf_path, "wb") as file:
    pdf_merger.write(file)

# remove all individual pdfs and pngs
for i, image_file in enumerate(image_files):
    os.remove(combined_path + f"/combined_image_{img_type}_{i}.pdf")
    os.remove(combined_path + f"/combined_image_{img_type}_{i}.png")

# Print the path to the combined PDF
print("The combined PDF is saved to:", combined_pdf_path)

# Store the order of images to disk
with open("test_data_collection/order.txt", "w") as f:
    for image_file in image_files:
        f.write(image_file.split('/')[-1][:-4].split('_')[1] + "\n")

# Extract Sketches from PDF

In [ ]:
writer = PdfWriter()
pdf_path = "combined_image_normals_modified.pdf"

with open(pdf_path, "rb") as file:

    pdf = PdfReader(file)

    # Initialize an empty list to store the extracted text
    extracted_text = []

    pages = pdf.pages

    for page_num in range(len(pdf.pages)):

        # numbers are hardcoded
        pdf.pages[page_num].mediabox.upper_right = (712, 455)
        pdf.pages[page_num].mediabox.lower_left = (437, 180)

        writer.add_page(pdf.pages[page_num])
        # write to document-output.pdf
        with open("temp.pdf", "wb") as fp:
            writer.write(fp)

        images = convert_from_path("temp.pdf")
        for i in range(len(images)):
            images[i].save('page'+ str(i) +'.png', 'PNG')
    